In [1]:
import os, io
from google.cloud import vision
import pandas as pd

c:\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Python310\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Python310\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"ServiceAccountToken.json"

client = vision.ImageAnnotatorClient()


In [3]:
'''
from cv2 import (VideoCapture, namedWindow, imshow, waitKey, destroyWindow, imwrite)
#cap = VideoCapture(0)

cam_port = 0
cam = VideoCapture(cam_port)


# reading the input using the camera
result, image = cam.read()
  
# If image will detected without any error, 
# show result
if result:
    # showing result, it take frame name and image 
    # output
    #imshow("photo", image)
  
    # saving image in local storage
    imwrite("photo.png", image)
  
    # If keyboard interrupt occurs, destroy image 
    # window
    waitKey(0)
    #destroyWindow("photo")
  
# If captured image is corrupted, moving to else part
else:
    print("No image detected. Please! try again")

'''
    

In [4]:
file_name = 'test2.png'
image_path = rf'C:\Users\BluRay\OneDrive\Desktop\GoogleVision\VisionAPIDemo\ReadingText\{file_name}'

with io.open(image_path, 'rb') as image_file:
    content = image_file.read()

# construct an iamge instance
image = vision.Image(content=content)

"""
# or we can pass the image url
image = vision.types.Image()
image.source.image_uri = 'https://edu.pngfacts.com/uploads/1/1/3/2/11320972/grade-10-english_orig.png'
"""

"\n# or we can pass the image url\nimage = vision.types.Image()\nimage.source.image_uri = 'https://edu.pngfacts.com/uploads/1/1/3/2/11320972/grade-10-english_orig.png'\n"

In [7]:
response = client.text_detection(image=image)  # returns TextAnnotation
df = pd.DataFrame(columns=['locale', 'description'])

texts = response.text_annotations
for text in texts:
    df = df.append(
        dict(
            locale=text.locale,
            description=text.description
        ),
        ignore_index=True
    )

text=df['description'][0]
print(text)

It
was the best of times,
it was the
of
worst of times,
times, it was the age
Wisdom, it was the age of foolishness


C:\Users\BluRay\AppData\Local\Temp\ipykernel_800\2135162917.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\BluRay\AppData\Local\Temp\ipykernel_800\2135162917.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\BluRay\AppData\Local\Temp\ipykernel_800\2135162917.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\BluRay\AppData\Local\Temp\ipykernel_800\2135162917.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(
C:\Users\BluRay\AppData\Local\Temp\ipykernel_800\2135162917.py:6: FutureWarning: The frame.append method is deprecated and will be r

In [ ]:
from gtts import gTTS
from langdetect import detect

mytext = text
lang = detect(text) 
audio = gTTS(text=mytext, lang=lang, slow=False)

audio.save("example.mp3")
os.system("start example.mp3")

In [5]:
import argparse
from enum import Enum
import io

from google.cloud import vision
from PIL import Image, ImageDraw

In [6]:
class FeatureType(Enum):
    PAGE = 1
    BLOCK = 2
    PARA = 3
    WORD = 4
    SYMBOL = 5

In [7]:
def draw_boxes(image, bounds, color):
    """Draw a border around the image using the hints in the vector list."""
    draw = ImageDraw.Draw(image)

    for bound in bounds:
        draw.polygon(
            [
                bound.vertices[0].x,
                bound.vertices[0].y,
                bound.vertices[1].x,
                bound.vertices[1].y,
                bound.vertices[2].x,
                bound.vertices[2].y,
                bound.vertices[3].x,
                bound.vertices[3].y,
            ],
            None,
            color,
        )
    return image

In [8]:
def get_document_bounds(image_file, feature):
    """Returns document bounds given an image."""
    client = vision.ImageAnnotatorClient()

    bounds = []

    with io.open(image_file, "rb") as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.document_text_detection(image=image)
    document = response.full_text_annotation

    # Collect specified feature bounds by enumerating all document features
    for page in document.pages:
        for block in page.blocks:
            for paragraph in block.paragraphs:
                for word in paragraph.words:
                    for symbol in word.symbols:
                        if feature == FeatureType.SYMBOL:
                            bounds.append(symbol.bounding_box)

                    if feature == FeatureType.WORD:
                        bounds.append(word.bounding_box)

                if feature == FeatureType.PARA:
                    bounds.append(paragraph.bounding_box)

            if feature == FeatureType.BLOCK:
                bounds.append(block.bounding_box)

    # The list `bounds` contains the coordinates of the bounding boxes.
    return bounds

In [9]:
def render_doc_text(filein):
    image = Image.open(filein)
    bounds = get_document_bounds(filein, FeatureType.BLOCK)
    draw_boxes(image, bounds, "blue")
    bounds = get_document_bounds(filein, FeatureType.PARA)
    draw_boxes(image, bounds, "red")
    bounds = get_document_bounds(filein, FeatureType.WORD)
    draw_boxes(image, bounds, "yellow")
    
    image.show()

In [10]:

render_doc_text(rf'C:\Users\BluRay\OneDrive\Desktop\GoogleVision\VisionAPIDemo\ReadingText\{file_name}')